<a href="https://colab.research.google.com/github/elianasv/HNC_PREDICTION/blob/main/FeatureSelection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This code performs feature selection using the Spearman Correlation matrix and Random Survival Forest as feature selector in the whole dataset**

Libraries downloading

In [1]:
!pip install --upgrade scikit-learn

In [2]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 7.0 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=8c7e7e20f4c6f62658613b665f838974dc364fac4c7290ae6ff7477f14ad65b4
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [3]:
!pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 31.3 MB/s eta 0:00:00


Libraries importing

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lifelines
from sksurv.ensemble import RandomSurvivalForest
from sklearn.inspection import permutation_importance

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Set the path of the folder in drive to save the generated datasets and outputs
folder_datasets = '/content/drive/MyDrive/Tesis/DefinitiveDatasets'
folder_outputs = '/content/drive/MyDrive/Tesis/DefinitiveOutputs'

In [7]:
# This URL leads to a Github repository containing the anonymized dataset with the random numbers already removed
filename = "https://github.com/elianasv/HNC_PREDICTION/raw/main/df_scaled.xlsx"
df_scaled = pd.read_excel(filename,engine='openpyxl')

In [ ]:
#df = pd.read_excel('/content/drive/MyDrive/Tesis/df_patiens_rem_reordered_renamed.xlsx') # Update file path

In [11]:
#Create a dataframe containing only the continuous variables
df_cont = df_scaled.drop(['DFS', 'DFSCensor','ID','OS','Dcd','LRC','LRCCensor','Centre','sex', 'chimio', 'stadeT', 'stadeN', 'stade', 'Tabac', 'P16','LocalisationORL'],axis=1)

### Correlation

In [19]:
# Calculate correlation matrix
correlation_matrix = df_cont.corr(method='spearman')

# Extract the upper triangle of the correlation matrix
upper = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Save the half matrix to an Excel file
upper.to_excel(f'{folder_outputs}/spearman_correlation_matrix_half.xlsx')


In [20]:
# Set a varying correlation threshold
threshold = 0.9

# Initialize a list to store all correlated features with correlation values for each feature
dropped_features_info = []

# Find highly correlated features and store all correlations for each feature to drop
for column in upper.columns:
    high_corr = upper[column][upper[column] > threshold]
    if not high_corr.empty:
        correlated_features = "; ".join([f"{idx}: {corr_value:.6f}" for idx, corr_value in high_corr.items()])
        dropped_features_info.append({
            'Feature to Drop': column,
            'Correlations': correlated_features
        })

# Convert the list of dropped features and their correlations into a DataFrame
df_dropped_features = pd.DataFrame(dropped_features_info)

# Drop highly correlated features from the original dataframe
df_corr = df_scaled.drop(columns=[item['Feature to Drop'] for item in dropped_features_info])

# Debugging: print the number of features remaining
print(f"Threshold: {threshold}, Features retained: {df_corr.shape[1]}")

# Display the DataFrame with dropped features and all their correlations
#print(df_dropped_features)

Threshold: 0.9, Features retained: 139


### Save the dataset

In [21]:
df_dropped_features.to_excel(f'{folder_outputs}/df_corr_removed_features.xlsx',index=False)

In [22]:
df_corr.to_excel(f'{folder_datasets}/df_corr.xlsx',index=False)

In [23]:
df_corr.to_csv(f'{folder_datasets}/df_corr.csv',index=False)

### Random Survival Forest (RSF)

In [9]:
#As the concordance method doesn't have its own c-index score calculator by default, this function is going to be passed
def concordance_index_scorer(estimator, X, y):
    """Custom scorer function for concordance index."""
    # Convert the structured array 'y' to separate arrays for time and event
    time = y['time']
    event = y['event']

    # Predict the risk scores
    risk_scores = estimator.predict(X)

    # Calculate the concordance index
    cindex = lifelines.utils.concordance_index(time, risk_scores, event_observed=event)

    return cindex


In [ ]:
# df_cont will be used to generate the survival data based on 'DFS' and 'DFSCensor'

# Prepare the survival data for RSF
# 'DFS' is the time in months, and 'DFSCensor' is the censoring indicator (1 = event, 0 = censored)
y = np.array([(bool(event), time) for event, time in zip(df_scaled['DFSCensor'], df_scaled['DFS'])],
             dtype=[('event', bool), ('time', float)])

# Initialize the Random Survival Forest model
rsf = RandomSurvivalForest(n_estimators=100, random_state=42)

# Fit RSF on the full dataset (without splitting the data)
rsf.fit(df_cont, y)

perm_importance = permutation_importance(rsf, df_cont, y, n_repeats=10, random_state=42, scoring=concordance_index_scorer)

# Create a DataFrame to rank the features by importance
importance_df = pd.DataFrame({
    'Feature': df_cont.columns,
    'Importance': perm_importance.importances_mean
}).sort_values(by='Importance', ascending=False)



In [27]:
# Select the top features based on importance
selected_features_rsf = importance_df.head(100)['Feature']  # Select the top 100 features
print(f'Selected features: {importance_df.head(100).shape[0]}')
selected_features_rsf.head(100).to_excel(f'{folder_outputs}/selected_features_rsf.xlsx',index=False)


Selected features: 100


In [25]:
#make a dataset with the noncontinuous features + selected features

# Create a list of non-continuous features
non_continuous_features = ['DFS', 'DFSCensor','ID','OS','Dcd','LRC','LRCCensor','Centre','sex', 'chimio', 'stadeT', 'stadeN', 'stade', 'Tabac', 'P16','LocalisationORL']

# Create a new dataset with the non-continuous features and the selected features from RSF
df_rsf_sel = df_scaled[non_continuous_features + list(selected_features_rsf)]


### Save the dataset

In [24]:
# Save the final dataset to an Excel file
df_rsf_sel.to_excel(f'{folder_datasets}/df_rsf_sel.xlsx', index=False)

In [18]:
df_rsf_sel.to_csv(f'{folder_datasets}/df_rsf_sel.csv',index=False)